In [ ]:
!pip install git+https://github.com/huggingface/peft
!pip install accelerate transformers einops datasets peft bitsandbytes
from kaggle_secrets import UserSecretsClient
import os
user_secrets = UserSecretsClient()
githubkey = user_secrets.get_secret("GITHUB")

!rm -rf phiver
command = f'''git clone https://{githubkey}@github.com/phuvinhnguyen/phiver.git'''
os.system(command)

In [2]:
import torch
from phiver.dataset.dataset import AICDataset, CombineAICDataset
from phiver.dataset.collator import VidLangCollator
from transformers import AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from phiver.dora.DoRA import DoRalizer
from phiver.model.phiver_configuration import *
from phiver.model.modeling_phiver import *
from peft import LoraConfig, get_peft_model

if torch.cuda.is_available():
    # Set the device to CUDA (GPU)
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    # Set the device to CPU
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")
    

# Load phi tokenizer
check_point = 'microsoft/phi-1_5'
tokenizer = AutoTokenizer.from_pretrained(check_point)

def count_parameters(model):
    print(f'trainable_params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')
    print(f'total_params: {sum(p.numel() for p in model.parameters())}')

# Change following paths to your own
TEST_CAPTIONS_DIR = '/kaggle/input/aic24-test/WTS_DATASET_PUBLIC_TEST/WTS_DATASET_PUBLIC_TEST/annotations/caption/test/public_challenge'
TEST_CAPTIONS_TRIM_DIR = '/kaggle/input/aic24-test/WTS_DATASET_PUBLIC_TEST/WTS_DATASET_PUBLIC_TEST/annotations/caption/test/public_challenge/normal_trimmed'
TEST_CAPTIONS_EXTERNAL_DIR = '/kaggle/input/aic24-test/WTS_DATASET_PUBLIC_TEST/WTS_DATASET_PUBLIC_TEST/external/BDD_PC_5K/annotations/caption/test/public_challenge'

TEST_VIDEOS_DIR = '/kaggle/input/aic24-videofeatures-test/matrix/train'
TEST_VIDEO_EXTERNAL_DIR = '/kaggle/input/aic24vidtestexternal/matrix/testexternal'

test_dataset = AICDataset(tokenizer=tokenizer, captions_path=TEST_CAPTIONS_DIR, videos_path=TEST_VIDEOS_DIR, is_test=True, device=device)
test_trim_dataset = AICDataset(tokenizer=tokenizer, captions_path=TEST_CAPTIONS_TRIM_DIR, videos_path=TEST_VIDEOS_DIR, is_test=True, device=device)
test_exte_dataset = AICDataset(tokenizer=tokenizer, captions_path=TEST_CAPTIONS_EXTERNAL_DIR, videos_path=TEST_VIDEO_EXTERNAL_DIR, is_test=True, dataset_type=1, device=device)
test_full_dataset = CombineAICDataset([test_dataset, test_trim_dataset, test_exte_dataset])

# Use DoRA for finetune llm
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'dense'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

config = PhiverConfig()
model = VideoPhiForCausalLM(config)
model.add_adapter(lora_config)

# Load checkpoint of the model (torch checkpoint)
model.load_state_dict(torch.load('/kaggle/input/aic24-phiver-checkpoint/original'))

count_parameters(model)

2024-03-22 16:10:20.624885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 16:10:20.624984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 16:10:20.749189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CUDA is available. Using GPU.


/kaggle/working/phiver/dataset/dataset.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_pedestrian['inputs_embeds'] = torch.tensor(output_pedestrian['event_embeds']).unsqueeze(0).to(self.device)
/kaggle/working/phiver/dataset/dataset.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_vehicle['inputs_embeds'] = torch.tensor(output_vehicle['event_embeds']).unsqueeze(0).to(self.device)


config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

trainable_params: 6754304
total_params: 1568489728


In [3]:
import json
from tqdm import tqdm

model.to(device).eval()

file_path = "./predict.json"
json_objects = []

collator = VidLangCollator(tokenizer, mlm=False)
collator.inference = True

batch_size = 8
start = 0

for index in tqdm(range(start, len(test_full_dataset), batch_size)):
    
    final_index = index + batch_size
    if final_index >= len(test_full_dataset):
        final_index = len(test_full_dataset)
    
    datas = [test_full_dataset[i] for i in range(index, final_index)]
    __ids = [data.pop('id') for data in datas]
    phases = [data['phase'] for data in datas]
    
    inputs = collator(datas)
    
    output = model.generate(**inputs, max_length=200)
    
    texts = tokenizer.batch_decode(output)
    
    json_objects = json_objects + [{
        'id': __id,
        'phase': phase.tolist(),
        'text': text
    } for __id, phase, text in zip(__ids, phases, texts)]
    
    with open(file_path, 'w') as json_file:
        json.dump(json_objects, json_file, indent=4)    

  0%|          | 0/747 [00:00<?, ?it/s]/kaggle/working/phiver/dataset/collator.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pad_raw = (torch.tensor(max_shape) - torch.tensor(tensor_shape)).tolist()
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 747/747 [1:27:23<00:00,  7.02s/it]


# Final output is saved to submit.json

In [4]:
import json
from typing import Dict
from typing_extensions import Tuple

with open('./predict.json', 'r') as rf:
    datas = json.load(rf)

mapping_phases = {'prerecognition': 0, 'recognition': 1, 'judgment': 2, 'action': 3, 'avoidance': 4, "0": 0, "1": 1, "2": 2, "3": 3, "4": 4}

def phase2token(phase: int):
    mapping_tokens = {i * 2 + j: (i, j) for i in range(5) for j in range(2)}
    mapping_objects = {0: 'pedestrian', 1: 'vehicle'}

    (label, i) = mapping_tokens[phase]

    return label, mapping_objects[i]

def get_correct_json(ids: Tuple, values: Dict):
    json_path = ids[-1]
    phase = ids[1]

    with open(json_path, 'r') as rf:
        phases = json.load(rf)['event_phase']

    for i in phases:
        if phase == int(i['labels'][0]):
            i.update(values)

            return ids[0], i

reformed_data = {}
final_data = {}

for data in datas:
    caption_path = data.pop('id')
    phase, _object = phase2token(data.pop('phase')[0][0])
    text = data.pop('text')

    # Get ID caption belongs to
    with open(caption_path, 'r') as rf:                                                                                                                                 
        json_data = json.load(rf)
        
        if 'video_name' in json_data: # External
            _id = (json_data['video_name'].split('.')[0], phase, caption_path)
        elif 'vehicle_view' in json_data: # Vehicle view
            _id = (os.path.splitext(json_data['vehicle_view'])[0], phase, caption_path)
        else:
            _id = (caption_path.split('/')[-3], phase, caption_path)

    # Update, Create result
    if _id not in reformed_data:
        reformed_data[_id] = {
            f'caption_{_object}': text
        }
    else:
        reformed_data[_id].update({
                                 f'caption_{_object}': text
                             })

for k, v in reformed_data.items():
    _id, value = get_correct_json(ids=k, values=v)

    if _id in final_data:
        final_data[_id].append(value)
    else:
        final_data[_id] = [value]

with open('submit.json', 'w') as wf:
    json.dump(final_data, wf, indent=4)